In [1]:
#!pip install ipynb
#%load_ext autoreload
#%autoreload 2
from ipynb.fs.full.facturxWidget import *

In [12]:
import cv2
import numpy as np
from PIL import Image
import io

style = {'description_width': '250px'}
layout = {'width': '500px'}

class ImageWidget(object):
    def __init__(self, image=None, display=True):
        self.count = 0
        
        self.image_w = widgets.Image(format='PNG')
        self.update_w = widgets.Checkbox(description='update',value=True)
        self.counter_w = widgets.Text(description = 'count',value = '', style=style, layout=layout,disabled=False)
        #VBox      = widgets.VBox([self.update_w,self.counter_w])  
        #self.HBox = widgets.HBox([self.image_w, VBox])  
        self.HBox = widgets.HBox([self.image_w])  
        
        self.setImage(image)
        if display == True:
            self.display()
    
    def display(self):
        #display(self.image_w)  
        display(self.HBox) 
    
    def getByteArray(self,image):    
        #return None
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()
        return img_byte_arr
    
    def evaluate(self, image=None):
        self.setImage(image)
        
    def _dict(self):
        return {'class' : self.__class__.__name__, 'parameters' : {}}
    
    def setImage(self,image):
        #print('ImageWidget')
        self.count += 1
        self.counter_w.value = str(self.count) 
        
        if self.update_w.value == False:
            return
        
        if image != None:
            try:
                #print('set image', type(image))
                if isinstance(image, dict):
                    image = image['image']
                self.image_w.value = self.getByteArray(image)    
            except Exception as exception: 
                print('Exception ImageWidget', str(exception))
            
    def getImage(self):
        return Image.open(io.BytesIO(self.image_w.value))

In [ ]:
#_widget._widget.__dict__

In [ ]:
import json

def getHostname(peer):
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    #user = 'User1'
    #user = 'hans'
    #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/User1@' + hostname + '/msp/'
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt,user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer),user=user)
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)  
            output = executePeerCmd(peer.pod_name, cmd)
            return output,_out
        except Exception as e:
            print(e)
            return output,_out
        
def query(peer, callopt, user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer),user=user)
            #hostname = getHostname(peer)
            #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + 'User1' + '@' + hostname + '/msp/'
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
            #strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0,'--cafile',path + 'cacerts/ca.zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' , '--certfile',path + 'signcerts/User1@zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' ,'--keyfile', path + 'keystore/priv_sk']
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)
            output = executePeerCmd(peer.pod_name, cmd)
            #output = executePeerCmd(peer.pod_name, cmd)           
            return json.loads(output),_out
        except Exception as e:
            print(e)
            return output,_out
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)       

def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   
    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
    
def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer'):
    #print(getArgList([role,name]))
    #callopt= '{"function":"AppendRole","Args":' + getArgList([role,name]) + '}'
    name = getSubmittingClientIdentity(peer, user=user)[:-1]
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role]) + '}'
    return invoke(peer,callopt)

def getRoles(peer,name=''):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":' + getArgList([name]) + '}'
    return query(peer,callopt)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)




## mockup

### load data

In [ ]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

invoices = load('invoices.json')
roles = load('roles.json')
roleTransactions = load('roleTransactions.json')

In [ ]:
def get_cert_cn(cert):
    return cert.split('=')[1].split('@')[0]

get_cert_cn(roles[0]['name']), roles[0]['roles']

In [ ]:
def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
    return result        
        
get_person_names_with_role('TaxInspector',roles)        

In [ ]:
roles

In [ ]:
if 'TaxInspector' in roles[0]['roles']:
    print()

In [ ]:
def invoke(peer,callopt,user='User1'):
    print(user,callopt)
    
def query(peer,callopt,user='User1'):
    print(user,callopt)    
    
class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name    
        
def addPeer(peername):  
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer        
  
def get_peer_of_user(name):
    return kubernetespeer(name=name)    

def addUser(peer,user):
    if peer.name in peerusers:
        peerusers[peer.name].append(user)
    else:
        peerusers[peer.name] = [user]

peers={}
peerusers ={}

addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 
addUser(peers['aubonmoulin'],'Tony.Dubois')    
addUser(peers['aubonmoulin'],'Stephanie.Hoarau')
addUser(peers['aubonmoulin'],'Alexandre.Payet')
addUser(peers['austriafactor'],'Emma.Steiner')    
addUser(peers['megustaolive'],'Pedro.Sanchez')  
addUser(peers['lyon-impots'],'Auguste.Boucher')  

peernames = list(peers.keys())

In [ ]:
peerusers
invoices[2]

## mockup

### load data

In [ ]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

invoices = load('invoices.json')
roles = load('roles.json')
roleTransactions = load('roleTransactions.json')

In [ ]:
def get_cert_cn(cert):
    return cert.split('=')[1].split('@')[0]

get_cert_cn(roles[0]['name']), roles[0]['roles']

In [ ]:
def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result            
    
get_person_names_with_role('TaxInspector',roles)        

In [ ]:
roles

In [ ]:
if 'TaxInspector' in roles[0]['roles']:
    print()

In [ ]:
def invoke(peer,callopt,user='User1'):
    print(user,callopt)
    
def query(peer,callopt,user='User1'):
    print(user,callopt)    
    
class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name    
        
def addPeer(peername):  
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer        
  
def get_peer_of_user(name):
    return kubernetespeer(name=name)    

def addUser(peer,user):
    if peer.name in peerusers:
        peerusers[peer.name].append(user)
    else:
        peerusers[peer.name] = [user]

peers={}
peerusers ={}

addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 
addUser(peers['aubonmoulin'],'Tony.Dubois')    
addUser(peers['aubonmoulin'],'Stephanie.Hoarau')
addUser(peers['aubonmoulin'],'Alexandre.Payet')
addUser(peers['austriafactor'],'Emma.Steiner')    
addUser(peers['megustaolive'],'Pedro.Sanchez')  
addUser(peers['lyon-impots'],'Auguste.Boucher')  

peernames = list(peers.keys())

In [ ]:
peerusers
invoices[2]

In [ ]:
#asset=listInvoices(peer=peers[peernames[2]], user='Tony.Dubois')[0][1]['Record']
asset=invoices[3] 
_widget = facturx_widgets(asset,'/', hideUnused=True) 
display(_widget)

In [ ]:
def create_dropdown_owner(_widget):
    _certs=[_widget.container.fields['owner'].widget.value]
    _certs = _certs + get_person_certs_with_role('Factor',roles)
    return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

def replace_owner_field_by_dropdown(_widget):
    _owners = create_dropdown_owner(_widget)
    _widget.container.fields['owner'] = Field(widget=_owners,path=_widget.container.fields['owner'].path)
    _widget.redisplay()

In [ ]:
owner=_widget.container.fields['owner'].widget.value.split(',')[0].split('=')[1].split('@')[0]
buyer=_widget.container.fields['buyer'].widget.value.split(',')[0].split('=')[1].split('@')[0]
taxinspector=get_person_names_with_role('TaxInspector',roles)
id=_widget.container.fields['ID'].widget.value
peer_buyer=get_peer_of_user(buyer)
peer_owner=get_peer_of_user(owner)
peer_taxinspector=get_peer_of_user(taxinspector)

In [ ]:
def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [ ]:
disable_all_fields()
enable_fields_for_role('Buyer')

In [ ]:
_widget.container.fields['received'].widget.value = True
_widget.container.fields['receivedOrder'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('Seller')

In [ ]:
# sold to factor
_widget.container.fields['owner'].widget.value = roles[2]['name']
_widget.container.fields['sold'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('Factor')

In [ ]:
_widget.container.fields['claimPaidBy'].widget.value = 'Christophe Monet'
_widget.container.fields['claimPaid'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('TaxInspector')

### role buyer

In [ ]:


peer=get_peer_of_user(buyer)

def received_changed(b):
    if b['name'] == 'value':
        receivedInvoice(peer_buyer, id=id, user=buyer)[1]
    
def receivedOrder_changed(b):
    if b['name'] == 'value':
        print('changed')
        receivedOrder(peer_buyer, id=id, user=buyer)[1]    
    
_widget.container.fields['received'].widget.observe(received_changed)
_widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)

### role owner / seller

In [ ]:

def sold_changed(b):
    if b['name'] == 'value':
        print('changed')
        #so(peer, id=id, user=seller)[1]
    
def claimPaid_changed(b):
    if b['name'] == 'value':
        print('changed')
        #receivedOrder(peer, id=id, user=seller)[1]    
    
_widget.container.fields['sold'].widget.observe(received_changed)
_widget.container.fields['claimPaid'].widget.observe(receivedOrder_changed)    

### role tax inspector

In [ ]:


def taxReceived_changed(b):
    if b['name'] == 'value':
        taxReceived(peer_taxinspector, id=id, user=taxinspector)[1]
    
_widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)

In [ ]:
receivedInvoice(peers[peernames[0]], id='asset14', user='Stephanie.Hoarau')[1]

# Rechnung

In [2]:
invoices = get_invoices()

In [6]:
outs = widgets.Output()
def create_field_container(x,path,hideUnused=True):
    with outs:
        clear_output()
        widget = facturx_widgets(x,path,hideUnused) 
        return widget
        
def mytest(x):    
    global _widget
    _widget = create_field_container(invoices.iloc[int(invoice.value)],pathtext.value, mytestfilter.value)  
        
#display(outs)

mytestfilter = widgets.Dropdown(options={True, False},value=True, description="FILTER", style=style, layout=layout)
invoice = widgets.Text(description = 'invoice',value = '15', style=style, layout=layout)
pathtext = widgets.Dropdown(options={'/exchangedDocumentContext', '/exchangedDocument', '/supplyChainTradeTransaction', 
                                     '/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty' , 
                                     '/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty',
                                    '/supplyChainTradeTransaction/applicableHeaderTradeAgreement'}, value = '/exchangedDocument',description="FILTER", style=style, layout=layout)
pathtext.observe(mytest, names='value')
invoice.on_submit(mytest)
display(invoice,mytestfilter,pathtext,outs)
create_field_container(invoices.iloc[int(invoice.value)],'/exchangedDocument', hideUnused=True)  

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Dropdown(description='FILTER', index=1, layout=Layout(width='500px'), options=(False, True), style=Description…

Dropdown(description='FILTER', layout=Layout(width='500px'), options=('/exchangedDocument', '/exchangedDocumen…

Output()

In [36]:
_widget = SimpleContainer(path='/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', asset=invoices.iloc[14], invoices=invoices)
#display(_widget)

Text(value='14', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

In [ ]:
#_widget._widget
_widget.asset, _widget._widget

In [44]:
filename = '/home/jovyan/persons/Emma.Steiner.png'
image = Image.open(open(filename, 'rb'))
#ratio = h / .image.size[1]
#newsize = (int(ratio * self.image.size[0]), int(ratio * self.image.size[1]) )
#self.image = self.image.resize([newsize[0],newsize[1]],Image.ANTIALIAS)
_imageWidget =  ImageWidget(image=image,display=False)

#HBox = widgets.HBox([_imageWidget.HBox, _widget.output]) 
HBox = widgets.HBox([_widget.output])
display(HBox)

In [45]:
_widget.redisplay()

In [9]:
!ls /home/jovyan/persons

 Alexandre.Payet.png	  Emma.Steiner.png	 Tony.Dubois.png
 Auguste.Boucher.png	  Pedro.Sanchez.png
'Christophe. Monet.png'   Stephanie.Hoarau.png
